# 3.1 지도학습

### 3.1.1 K-최근접 이웃(K-nearest neighbor)
새로운 입력을 받았을 때, 기존 클러스터에서 모든 데이터와 인스턴스 기반 거리를 측정해서 가장 많은 속성을 갖는 클러스터에 할당하는 분류 알고리즘
- 기존 데이터들과 하나씩 거리를 계산하고 거리상 가장 가까운 데이터 k개를 선택하여 해당 클러스터에 할당
- k값에 따라 분류결과가 달라짐

In [3]:
# 라이브러리 호출 및 데이터 준비
import numpy as np                  # 벡터 및 행렬 연산 처리
import matplotlib.pyplot as plt     #  차트나 플롯을 그리는 라이브러리
import pandas as pd                 # 데이터 분석 및 조작을 위한 라이브러리
from sklearn import metrics         # 모델 성능 평가

names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']

dataset = pd.read_csv('data/chap3/data/iris.data', names=names)

In [5]:
dataset

,sepal-length,sepal-width,petal-length,petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [20]:
# 훈련과 검증 데이터셋 분리
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)

In [21]:
# 모델 생성 및 훈련
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=50)

In [22]:
# 모델 정확도
from sklearn.metrics import accuracy_score
y_pred = knn.predict(X_test)
print("정확도: {}".format(accuracy_score(y_test, y_pred)))

정확도: 0.8666666666666667


In [23]:
# 최적 K 찾기
k = 10
acc_array = np.zeros(k)
# k값 돌려가면서 정확도 저장
for k in np.arange(1, k+1, 1):
    classifier = KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    acc_array[k-1] = acc
    
print(acc_array)
# 최대값 산출    
max_acc = np.amax(acc_array)
acc_list = list(acc_array)
k = acc_list.index(max_acc)
print("정확도", max_acc, "으로 최적의 k는", k+1, "입니다.")

[0.9 1.  1.  1.  1.  1.  1.  1.  1.  1. ]
정확도 1.0 으로 최적의 k는 2 입니다.


---
### 3.1.2 서포트 벡터 머신 (Supprot Vector Machine, SVM)
분류를 위한 기준선을 정의하는 모델로 새로운 데이터가 나타나면 결정 경계(기준선)를 기준으로 경계의 어느 쪽에 속하는지 분류하는 모델
- 결정 경계는 데이터가 분류된 클래스에서 최대한 멀리 떨어져 있을 때 가장 좋다.
- 마진(margin)
    - 결정경계와 서포트 벡터 사이의 거리
- 서포트 벡터(support vector)
    - 결정 경계와 가까이 있는 데이터
- 최적의 결정 경계는 마진을 최대로 해야한다.
- 최적 결정 경계를 위해 마진을 최대로 해야하는데, 이때 이상치(outlier)를 다루는 것이 중요
    - 하드 마진(hard margin) : 이상치를 허용하지 않은 마진
    - 소프트 마진 (sorft margin) : 이상치들이 마진 안에 포함되는 것을 허용한 마진


In [3]:
!pip install watermark

In [5]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.0.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
anaconda-client               1.9.0
anaconda-navigator            2.1.4
anaconda-project              0.10.2
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.6.6
astropy                       5.0.4
asttokens                     2.0.5
astunparse                    1.6.3
async-timeout                 4.0.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backpor

python-lsp-jsonrpc            1.0.0
python-lsp-server             1.2.4
python-slugify                5.0.2
python-snappy                 0.6.0
pytz                          2021.3
pyviz-comms                   2.0.2
PyWavelets                    1.3.0
pywin32                       302
pywin32-ctypes                0.2.0
pywinpty                      2.0.2
PyYAML                        6.0
pyzmq                         22.3.0
QDarkStyle                    3.0.2
qstylizer                     0.1.10
QtAwesome                     1.0.3
qtconsole                     5.3.0
QtPy                          2.0.1
queuelib                      1.5.0
regex                         2022.3.15
requests                      2.27.1
requests-file                 1.5.1
requests-oauthlib             1.3.1
rope                          0.22.0
rsa                           4.7.2
Rtree                         0.9.7
ruamel-yaml-conda             0.15.100
s3transfer                    0.5.0
scikit-image        

In [6]:
from sklearn import svm
from sklearn import metrics
from sklearn import datasets
from sklearn import model_selection
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

TF_CPP_MIN_LOG_LEVEL
- 환경변수. 로깅을 제어한다.
- 기본값은 0으로 모든 로그가 표시
- 1 : INFO 로그 필터링
- 2 : WARNING 로그를 필터링
- 3 : ERROR 로그를 필터링

In [ ]:
ㅇ